In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt

os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/lib/cuda'
os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [2]:
import tensorflow as tf
from tensorflow import keras

print("GPU available:", len(tf.config.list_physical_devices('GPU')))

2023-05-28 20:52:37.273184: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 20:52:38.179434: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU available: 1


2023-05-28 20:52:39.273359: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-28 20:52:39.313982: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-28 20:52:39.315827: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
from data_preprocessing import DataProcessor, Dataset
from model import OCCModel
from autoencoder import MLPAutoencoder
from var_autoencoder import MLPVarAutoencoder
from loss import WeightedMSE

In [4]:
PROJECT_PATH = '/raid/shumnov/music-style-performer/'

In [5]:
architecture = [((256, 128), (0, 0), (128, 256), (0, 0)), 
                ((256, 128, 128), (0.1, 0.1, 0.1), (128, 128, 256), (0.1, 0.1, 0.1)), 
                ((256, 256, 128), (0.4, 0.2, 0.1), (128, 256, 256), (0.1, 0.1, 0.1)),
                ((128, 64, 64), (0, 0, 0), (64, 64, 128), (0, 0, 0))]
noise = [0.02, 0.08, 0.2, 0.4]
latent_dim = [64, 32]
absolute_velocities = [True, False]

In [6]:
def test(av, ld, n, arch, name, legw=1, checkpoint_id=None, epochs=10):
    print(name)
    
    dp = DataProcessor(
        notes_qty=64, 
        include_first_tone=False, 
        absolute_velocities=av)
    
    dset = Dataset(dp, 64)
    
    print('Loading dataset...')
    dset.load_dset(PROJECT_PATH + 'data/dset64.npy', 2000)
    print('Dataset loaded')
    
    autoencoder = MLPAutoencoder(
        input_size=dp.input_size,
        latent_dim=ld,
        noise=n,
        encoder_layers=arch[0],
        encoder_dropout=arch[1],
        decoder_layers=arch[2],
        decoder_dropout=arch[3])
    
    loss = WeightedMSE(
        vel_mask=dp.vel_mask, 
        leg_mask=dp.leg_mask, 
        first_last_mask=dp.first_last_mask, 
        vel_weight=1, 
        leg_weight=legw,
        first_last_weight=1)
    
    model = OCCModel(
        autoencoder=autoencoder,
        dataprocessor=dp,
        dist_weight=1, 
        vel_weight=1, 
        leg_weight=legw
    )
    model.compile(
        modelsdir=PROJECT_PATH+'traindata/ae/64',
        name=name, 
        optimizer=keras.optimizers.Adam(3e-5),
        loss=loss,
        ckpt_epochs=1
    )
    model.summary()
    
    if checkpoint_id is not None:
        model.restore(checkpoint_id)
    
    print('Start training...')
    model.fit(dset, epochs, 3)

In [ ]:
for ld in range(len(latent_dim)):
    for av in range(len(absolute_velocities)):
        for n in range(len(noise)):
            for a in range(len(architecture)):
                test(absolute_velocities[av], latent_dim[ld], noise[n], architecture[a], f'{ld}{av}{n}{a}')

In [ ]:
test(absolute_velocities[0], latent_dim[0], noise[3], architecture[0], f'0030', legw=1, epochs=40)

0023
Loading dataset...


/raid/shumnov/anaconda3/envs/tf/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:411: RuntimeWarning: Mean of empty slice.
  loc = data.mean()
/raid/shumnov/anaconda3/envs/tf/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/raid/shumnov/anaconda3/envs/tf/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:416: RuntimeWarning: Mean of empty slice.
  scale = np.sqrt(((data - loc)**2).mean())


Dataset loaded


2023-05-28 20:53:20.434395: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-28 20:53:20.436280: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-28 20:53:20.437938: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Start training...
Epoch 1/40


2023-05-28 20:53:44.694846: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x21be3800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-28 20:53:44.694893: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla V100-SXM3-32GB, Compute Capability 7.0
2023-05-28 20:53:44.700687: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-28 20:53:44.933380: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-28 20:53:45.021054: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2023-05-28 20:53:45.071170: I ./tensorflow/compiler/jit/device_compiler.h:180] Com

82301/82301 - 277s - loss: 0.2516 - val_loss: 0.1991 - occ_accuracy: 0.2737 - 277s/epoch - 3ms/step
Epoch 2/40
